### モデルの比較をしてみた

In [1]:
# ボストン・ハウジングデータの読み込み
import pandas as pd
from IPython.core.display import display
from sklearn.datasets import load_boston

dataset = load_boston()
print('dataset.keys():',dataset.keys())
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.Series(dataset.target, name='y')

# 中身の確認
print('------------------------------')
print(f'X shape:{X.shape}')
print('------------------------------')
display(X.join(y).head())

print(dataset.DESCR)

dataset.keys(): dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])
------------------------------
X shape:(506, 13)
------------------------------


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,y
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

### パイプラインを使用し各モデルを一気に比較
- ols  :Ordinary Least Squares（最小2乗回帰）
- ridge:Ridge Regression（リッジ回帰）
- tree :Decision Tree（決定木）
- rf   :Random Forest（ランダムフォレスト）
- gbr1 :Gradient Boosting（勾配ブースティングr1）
- gbr2 :Gradient Boosting(勾配ブースティングr2)

In [2]:
# ライブラリの読み込み
# 前処理
from sklearn.preprocessing import StandardScaler #標準化
# モデル
from sklearn.linear_model import LinearRegression, Ridge #線形回帰とリッジ回帰
from sklearn.tree import DecisionTreeRegressor #決定木
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor #ランダムフォレストと勾配ブースティング
from sklearn.pipeline import Pipeline
# 評価
from sklearn.model_selection import train_test_split #訓練テスト分割
from sklearn.metrics import r2_score

In [3]:
# ホールドアウト
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

# pipelineにセット
pipelines = {
    'ols': Pipeline([('scl', StandardScaler()),
                     ('est', LinearRegression())]),
    'ridge': Pipeline([('scl', StandardScaler()),
                       ('est', Ridge(random_state=0))]),
    'tree': Pipeline([('scl', StandardScaler()),
                      ('est', DecisionTreeRegressor(random_state=0))]),
    'rf': Pipeline([('scl', StandardScaler()),
                    ('est', RandomForestRegressor(random_state=0))]),
    'gbr1': Pipeline([('scl', StandardScaler()),
                      ('est', GradientBoostingRegressor(n_estimators=100, random_state=0))]),
    'gbr2': Pipeline([('scl', StandardScaler()),
                      ('est', GradientBoostingRegressor(n_estimators=250, random_state=0))])
}

### 学習と評価をパイプラインで一気に行う

In [4]:
# piplinesはディキショナリー名なので他の名前でも良い
# Pipleineはメソッド、keyが名前、バリューがパイプライン

# dict型に評価を入れていく
scores = {}
for pipe_name, pipeline in pipelines.items():# items(): 各要素のキーkeyと値valueに対してforループ処理
    pipeline.fit(X_train, y_train)# 標準化の後学習
    scores[(pipe_name, 'train')] = r2_score(y_train, pipeline.predict(X_train))
    scores[(pipe_name, 'test')] = r2_score(y_test, pipeline.predict(X_test))
    
pd.Series(scores).unstack()
# 訓練データとテストデータの差が広いと過学習している可能性が高い

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


,test,train
gbr1,0.924750,0.977915
gbr2,0.925818,0.994840
ols,0.763417,0.729359
rf,0.884060,0.977035
ridge,0.763404,0.729336
tree,0.821282,1.000000
